In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openpyxl
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
data = pd.read_excel('Indicadores.xlsx')
data = data.sort_values(by='Periodo')
data = data.reset_index(drop=True)

In [ ]:
data.index = pd.to_datetime(data['Periodo'])
data.drop(columns='Periodo',inplace=True)
data.head()
data

In [ ]:
data.plot()

In [ ]:
def check_for_stationary(data):
	#X = data['Valor'].values
	X = data.values
	result = adfuller(X)
	if result[1] <= 0.05:
		print('the variable is stationary')
	else:
		print('the variable is non starionary')
	print('p_value: '+ str(result[1]))
	return

In [ ]:
check_for_stationary(data['Valor'])

In [ ]:
an_change = np.log(data['Valor']) - np.log(data['Valor'].shift(12))
an_change = an_change.dropna()
check_for_stationary(an_change)

In [ ]:
sm.graphics.tsa.plot_acf(an_change.values.squeeze(), lags=12)
plt.show()

In [ ]:
sm.graphics.tsa.plot_pacf(an_change.values.squeeze(), lags=12, method="ywm")
plt.show()

Nota: pacf muestra la autocorrelacion parcial entre la serie y los lags

La grafica ACF muestra un decaimiento lento, mientras que la pacf muestra uno muy rapido (inicia con autocorrelacion muy alta en los primeros 2 lags y luego baja hasta cero y negativos). Esto nos dice que el modelo es un AR. 

Determinar la cantidad de AR (p):
- Se determina a partir de los lags significativos en el PACF 
- p=2    q=0

D y d: 

- Debido a que estoy modelando la diferencia de seasonal (crecimiento anual) tenemos un D=1 y d=0 porque solo se necesitó sacar la 1era diferencia seasonal para volver "stationary" nuestro modelo.

P y Q = 0

we are modelling the seasonal difference of the log, which is the ANNUAL % GROWTH of passengers month by month.

In [ ]:
model = SARIMAX(np.log(data['Valor']), order=(2,0,0),seasonal_order =(0,1,0,12),trend='c',simple_differencing=True)
history = model.fit()
history.summary()

El crecimiento anual del mes pasado impacta en un 75% al crecimiento anual en este mes. El crecimiento anual de hace 2 meses casi no tiene impacto en el crecimiento anual en este mes, ya que solo se relacionan en un 4%.
Dicho de otro modo, por cada porciento del crecimiento anual del mes anterior, se espera que el crecimiento anual de este mes crezca un 75%

El drift es positivo y muy significativo en el modelo, por lo que se puede notar que el valor crezca con el tiempo

Ahora quiero conocer los residuales del modelo, para saber si aun hay algo que se puede tomar en consideración dentro del modelo para mejorar su desempeño. 

In [ ]:
residuals = history.resid
print("aic = "+ str(history.aic))
sm.graphics.tsa.plot_acf(residuals.squeeze(),lags=24)
plt.show()

Es claro que aun hay algo que no se está tomando en cuenta en el modelo, y gracias a la gráfica puedo ver que hay un pico en el lag 12, por lo que debería aumentar un término seasonal anual al modelo

In [ ]:
model2 = SARIMAX(np.log(data['Valor']), order=(2,0,0),seasonal_order =(0,1,1,12),trend='c',simple_differencing=True)
history2 = model2.fit()

In [ ]:
residuals2 = history2.resid
print("aic = "+ str(history2.aic))
sm.graphics.tsa.plot_acf(residuals2.squeeze(),lags=24)
plt.show()

Si volvemos a ver los residuales notamos que el pico ha desaparecido, lo cual significa que hemos logrado ajustar el modelo

In [ ]:
history2.summary()

Coeficientes:
- drift: es sumamente significativo (z = -4.368) y está relacionado con el crecimiento económico anual en un -0.22% 
- phi 1: tiene alta significancia en el modelo (z=22.477) y está relacionado con el crecimiento económico anual en un 76% , lo que significa que el 76% del crecimiento anual se puede explicar gracias a este coeficiente
- phi 2: tiene alta significancia en el modelo (z=3.611) y está relacionado con el crecimiento económico anual en un 12% , lo que significa que el 12% del crecimiento anual se puede explicar gracias a este coeficiente

## Predicciones

In [ ]:
data.shape
len(data)

In [ ]:
modelFinal = SARIMAX((data['Valor']), order=(2,0,0),seasonal_order =(0,1,1,12),trend='c',simple_differencing=True)
historyFinal = modelFinal.fit()
historyFinal.summary()

In [ ]:
y = np.log(data['Valor'])
y_pred = 0.2520 + 0.7571*data['Valor'].shift(1) + 0.1*data['Valor'].shift(1) - 0.8121*(-0.5/0.8121)
data['Pred'] = y_pred

In [ ]:
data[['Valor','Pred']].plot(figsize=(12,8))